<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import InputLayer, Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.models import Model

In [ ]:
dataset, dataset_info = tfds.load("malaria", with_info=True, as_supervised=True, shuffle_files=True, split=["train"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteSXYR01/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):

  SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * SIZE))
  val_dataset = dataset.skip(int(TRAIN_RATIO * SIZE)).take(int(VAL_RATIO * SIZE))
  test_dataset = dataset.skip(int((TRAIN_RATIO + VAL_RATIO) * SIZE))

  return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

In [ ]:
IM_SIZE = 224

def resize_rescale(image, label, image_size = IM_SIZE):
  return tf.image.resize(image, (image_size, image_size)) / 255.0, label

In [ ]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
lenet_loaded_model = tf.keras.models.load_model("lenet.hdf5")
lenet_loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_6 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_7 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [ ]:
lenet_loaded_model.evaluate(test_dataset)

87/87 [==============================] - 20s 33ms/step - loss: 0.1754 - accuracy: 0.9427


[0.17544764280319214, 0.9426705241203308]

<h2> <b> Callbacks </b> </h2>

In [ ]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler

<b> Early Stopping callback </b>

In [ ]:
es_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
lenet_loaded_model.compile(
    optimizer = Adam(learning_rate= 0.05),
    loss = BinaryCrossentropy(),
    metrics = "Accuracyi"
    )

In [ ]:
history = lenet_loaded_model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

Epoch 1/3


TypeError: ignored

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters= 6, kernel_size= 5, strides= 1, padding= "valid",activation= "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size= 2, strides= 2),

    Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation="relu"),
    BatchNormalization(),

    Dense(10, activation="relu"),
    BatchNormalization(),

    Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       456       
                                                                 
 batch_normalization (Batch  (None, 220, 220, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_1 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.08),
              loss=BinaryCrossentropy(),
              metrics="accuracy")

In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs=15, verbose=1, callbacks=[es_callback])

Epoch 1/15
603/603 [==============================] - 48s 69ms/step - loss: 0.6120 - accuracy: 0.6636 - val_loss: 3.7800 - val_accuracy: 0.6512
Epoch 2/15
603/603 [==============================] - 42s 69ms/step - loss: 0.2613 - accuracy: 0.9110 - val_loss: 0.2418 - val_accuracy: 0.9418
Epoch 3/15
603/603 [==============================] - 43s 71ms/step - loss: 0.2022 - accuracy: 0.9378 - val_loss: 0.3910 - val_accuracy: 0.9211
Epoch 4/15
603/603 [==============================] - 41s 68ms/step - loss: 0.1936 - accuracy: 0.9413 - val_loss: 0.3492 - val_accuracy: 0.9349
Epoch 5/15
603/603 [==============================] - 41s 68ms/step - loss: 0.1814 - accuracy: 0.9464 - val_loss: 0.2754 - val_accuracy: 0.9312
Epoch 5: early stopping


<h2> <b> Learning Rate Scheduling </b> </h2>

In [ ]:
def scheduler(epoch, lr):
  if epoch < 3:
    print(f"current learning rate: {lr}\n")
    return lr
  else:
    print(f"after 10 epochs learning rate: {lr}\n")
    return lr * tf.math.exp(-0.1)

scheduler_callback = LearningRateScheduler(scheduler, verbose=1)

In [ ]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs=15, verbose=1, callbacks=[scheduler_callback])

current learning rate: 0.07999999821186066


Epoch 1: LearningRateScheduler setting learning rate to 0.07999999821186066.
Epoch 1/15
603/603 [==============================] - 48s 79ms/step - loss: 0.1687 - accuracy: 0.9492 - val_loss: 0.2265 - val_accuracy: 0.9436 - lr: 0.0800
current learning rate: 0.07999999821186066


Epoch 2: LearningRateScheduler setting learning rate to 0.07999999821186066.
Epoch 2/15
603/603 [==============================] - 49s 82ms/step - loss: 0.1627 - accuracy: 0.9510 - val_loss: 0.7725 - val_accuracy: 0.9194 - lr: 0.0800
current learning rate: 0.07999999821186066


Epoch 3: LearningRateScheduler setting learning rate to 0.07999999821186066.
Epoch 3/15
603/603 [==============================] - 41s 68ms/step - loss: 0.1486 - accuracy: 0.9540 - val_loss: 0.1668 - val_accuracy: 0.9532 - lr: 0.0800
after 10 epochs learning rate: 0.07999999821186066


Epoch 4: LearningRateScheduler setting learning rate to 0.07238698750734329.
Epoch 4/15
603/603 [=============